In [43]:
import numpy as np
import pandas as pd
import tqdm, glob
from pybel import *
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from sympy.geometry import Point3D

In [27]:
# initiate the plotly notebook mode
init_notebook_mode(connected=True)


def plot_molecule(molecule_name, structures_df):
    """Creates a 3D plot of the molecule"""
    
    atomic_radii = dict(C=0.77, F=0.71, H=0.38, N=0.75, O=0.73)  
    cpk_colors = dict(C='black', F='green', H='white', N='blue', O='red')

    molecule = structures_df[structures_df.molecule_name == molecule_name]
    coordinates = molecule[['x', 'y', 'z']].values
    x_coordinates = coordinates[:, 0]
    y_coordinates = coordinates[:, 1]
    z_coordinates = coordinates[:, 2]
    elements = molecule.atom.tolist()
    radii = [atomic_radii[element] for element in elements]
    
    def get_bonds():
        """Generates a set of bonds from atomic cartesian coordinates"""
        ids = np.arange(coordinates.shape[0])
        bonds = dict()
        coordinates_compare, radii_compare, ids_compare = coordinates, radii, ids
        
        for _ in range(len(ids)):
            coordinates_compare = np.roll(coordinates_compare, -1, axis=0)
            radii_compare = np.roll(radii_compare, -1, axis=0)
            ids_compare = np.roll(ids_compare, -1, axis=0)
            distances = np.linalg.norm(coordinates - coordinates_compare, axis=1)
            bond_distances = (radii + radii_compare) * 1.3
            mask = np.logical_and(distances > 0.1, distances <  bond_distances)
            distances = distances.round(2)
            new_bonds = {frozenset([i, j]): dist for i, j, dist in zip(ids[mask], ids_compare[mask], distances[mask])}
            bonds.update(new_bonds)
        return bonds            
            
    def atom_trace():
        """Creates an atom trace for the plot"""
        colors = [cpk_colors[element] for element in elements]
        markers = dict(color=colors, line=dict(color='lightgray', width=2), size=7, symbol='circle', opacity=0.8)
        trace = go.Scatter3d(x=x_coordinates, y=y_coordinates, z=z_coordinates, mode='markers', marker=markers,
                             text=elements, name='')
        return trace

    def bond_trace():
        """"Creates a bond trace for the plot"""
        trace = go.Scatter3d(x=[], y=[], z=[], hoverinfo='none', mode='lines',
                             marker=dict(color='grey', size=7, opacity=1))
        for i, j in bonds.keys():
            trace['x'] += (x_coordinates[i], x_coordinates[j], None)
            trace['y'] += (y_coordinates[i], y_coordinates[j], None)
            trace['z'] += (z_coordinates[i], z_coordinates[j], None)
        return trace
    
    bonds = get_bonds()
    
    zipped = zip(range(len(elements)), x_coordinates, y_coordinates, z_coordinates)
    annotations_id = [dict(text=num, x=x, y=y, z=z, showarrow=False, yshift=15, font = dict(color = "blue"))
                   for num, x, y, z in zipped]
    
    annotations_length = []
    for (i, j), dist in bonds.items():
        p_i, p_j = Point3D(coordinates[i]), Point3D(coordinates[j])
        p = p_i.midpoint(p_j)
        annotation = dict(text=dist, x=float(p.x), y=float(p.y), z=float(p.z), showarrow=False, yshift=15)
        annotations_length.append(annotation)   
    
    updatemenus = list([
        dict(buttons=list([
                 dict(label = 'Atom indices',
                      method = 'relayout',
                      args = [{'scene.annotations': annotations_id}]),
                 dict(label = 'Bond lengths',
                      method = 'relayout',
                      args = [{'scene.annotations': annotations_length}]),
                 dict(label = 'Atom indices & Bond lengths',
                      method = 'relayout',
                      args = [{'scene.annotations': annotations_id + annotations_length}]),
                 dict(label = 'Hide all',
                      method = 'relayout',
                      args = [{'scene.annotations': []}])
                 ]),
                 direction='down',
                 xanchor = 'left',
                 yanchor = 'top'
            ),        
    ])
    
    data = [atom_trace(), bond_trace()]
    axis_params = dict(showgrid=False, showbackground=False, showticklabels=False, zeroline=False, titlefont=dict(color='white'))
    layout = dict(scene=dict(xaxis=axis_params, yaxis=axis_params, zaxis=axis_params, annotations=annotations_id), 
                  margin=dict(r=0, l=0, b=0, t=0), showlegend=False, updatemenus=updatemenus)

    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

# Read necessary files

In [3]:
struct = pd.read_csv('../data/structures.csv')
train = pd.read_csv('../data/train.csv')
tes = pd.read_csv('../data/test.csv')

In [4]:
struct.head(3)

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277


In [5]:
train.head(3)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548


# Pybel MVP

In [51]:
mol_file = '../data/structures/dsgdb9nsd_005567.xyz'

In [52]:
plot_molecule(mol_file.split('/')[-1].split('.')[0], struct)

In [63]:
%%time
for mol in readfile('xyz', mol_file):
    for atm in mol:
        ix = atm.idx - 1
        bonds = []
        for bond in OBAtomBondIter(atm.OBAtom):
            beg_ix = bond.GetBeginAtomIdx() - 1
            end_ix = bond.GetEndAtomIdx() - 1
            bond_order = bond.GetBondOrder()
            bonds.append(f'{bond_order:d}-bond | {beg_ix:d}-{end_ix:d}')
        print(ix, bonds)

0 ['1-bond | 9-0', '1-bond | 8-0', '1-bond | 0-1', '1-bond | 0-10']
1 ['1-bond | 11-1', '1-bond | 0-1', '1-bond | 1-2', '1-bond | 1-12']
2 ['1-bond | 7-2', '1-bond | 1-2', '2-bond | 2-3']
3 ['2-bond | 2-3', '1-bond | 5-3', '1-bond | 3-4']
4 ['1-bond | 3-4', '1-bond | 4-15', '1-bond | 4-14', '1-bond | 4-13']
5 ['1-bond | 6-5', '1-bond | 5-16', '1-bond | 5-3']
6 ['1-bond | 17-6', '2-bond | 7-6', '1-bond | 6-5']
7 ['2-bond | 7-6', '1-bond | 7-2']
8 ['1-bond | 8-0']
9 ['1-bond | 9-0']
10 ['1-bond | 0-10']
11 ['1-bond | 11-1']
12 ['1-bond | 1-12']
13 ['1-bond | 4-13']
14 ['1-bond | 4-14']
15 ['1-bond | 4-15']
16 ['1-bond | 5-16']
17 ['1-bond | 17-6']
CPU times: user 4.13 ms, sys: 5 µs, total: 4.14 ms
Wall time: 3.3 ms
